In [16]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_json('train.json')

In [4]:
test = pd.read_json('test.json')

In [5]:
train.shape

(49352, 15)

In [20]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [6]:
from sklearn.preprocessing import LabelEncoder
train['interest'] = LabelEncoder().fit_transform(train.interest_level)

In [7]:
y = train['interest']
train.drop(['interest','interest_level'], axis =1, inplace = True)

In [8]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score,roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split ,GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.svm import SVC

In [11]:
#y = train['interest']
X = train[['bathrooms','bedrooms','price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 10)
cvec = TfidfVectorizer(ngram_range =(2,3),analyzer=u'word', stop_words='english')
cvec.fit(X_train)
X_train  = pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())


X_test = pd.DataFrame(cvec.transform(X_test).todense(), columns=cvec.get_feature_names())

rf = RandomForestClassifier(n_jobs=-1, random_state=10)
rf_fit = rf.fit(X_train,y_train)

y_pred = rf_fit.predict(X_test)
probabilities = rf_fit.predict_proba(X_test)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 10)
rf = RandomForestClassifier(n_jobs=-1, random_state=10)
rf_fit = rf.fit(X_train,y_train)
y_pred = rf_fit.predict(X_test)
probabilities = rf_fit.predict_proba(X_test)
y_score=[]
for i in range(0,len(probabilities)):
    y_score.append(probabilities[i][1])



conmat = np.array(confusion_matrix(y_test, y_pred, labels=[2,1,0]))

confusion = pd.DataFrame(conmat, index=['2','1', '0'],
                         columns=['predicted_2','predicted_1','predicted_0'])

print('confusion_matrix:')
print(confusion)
print('classification_report:')
print(classification_report(y_test, y_pred))
print('accuracy_score:')
print(accuracy_score(y_test, y_pred))
print('auc_score:')
print(roc_auc_score(y_test, y_score))

confusion_matrix:
   predicted_2  predicted_1  predicted_0
2          253         1961           69
1          333         6475           71
0           91          527           91
classification_report:
             precision    recall  f1-score   support

          0       0.39      0.13      0.19       709
          1       0.72      0.94      0.82      6879
          2       0.37      0.11      0.17      2283

avg / total       0.62      0.69      0.62      9871

accuracy_score:
0.690811467936
auc_score:


ValueError: multiclass format is not supported